In [1]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [2]:
### 시각화를 위한 라이브러리
# - 파이썬 시각화에서 가장 기본적인 시각화 라이브러리
import matplotlib.pyplot as plt

### 한글처리
from matplotlib import font_manager, rc

### 운영체제 확인 라이브러리
import platform

### 시각화 시 마이너스(-, 음수) 기호 깨짐 방지
plt.rcParams["axes.unicode_minus"] = False

### OS별 한글처리
# - 윈도우 운영체게
if platform.system() == "Windows" :
    # path = "c:/Windows/Fonts/malgun.ttf"
    # font_name = font_manager.FontProperties(fname=path).get_name()
    # rc("font", family = font_name)
    
    ### 또는 아래처럼 한줄로도 가능 (아래 한글처리를 주로 사용합니다.)
    plt.rc("font", family = "Malgun Gothic")

# - Mac 운영체제
elif platform.system() == "Darwin" :
    rc("font", family = "AppleGothic")
    
else :
    print("넌 누구?")

In [3]:
labeled = pd.read_csv("./data/labeled_data.csv")

In [4]:
labeled_cn7 = labeled[(labeled["PART_NAME"] == "CN7 W/S SIDE MLD'G RH") |(labeled["PART_NAME"] == "CN7 W/S SIDE MLD'G LH")]

In [5]:
labeled_cn7 = labeled_cn7.drop(columns=["_id", "TimeStamp","PART_FACT_PLAN_DATE", "Reason", "PART_FACT_SERIAL", "PART_NAME",
               "EQUIP_NAME", "EQUIP_CD", "Switch_Over_Position", "Mold_Temperature_1","Mold_Temperature_2","Mold_Temperature_5",
                "Mold_Temperature_6", "Mold_Temperature_7","Mold_Temperature_8","Mold_Temperature_9","Mold_Temperature_10",
               "Mold_Temperature_11","Mold_Temperature_12","Barrel_Temperature_7"])

In [6]:
labeled_cn7

,PassOrFail,Injection_Time,Filling_Time,Plasticizing_Time,Cycle_Time,Clamp_Close_Time,Cushion_Position,Plasticizing_Position,Clamp_Open_Position,Max_Injection_Speed,...,Average_Back_Pressure,Barrel_Temperature_1,Barrel_Temperature_2,Barrel_Temperature_3,Barrel_Temperature_4,Barrel_Temperature_5,Barrel_Temperature_6,Hopper_Temperature,Mold_Temperature_3,Mold_Temperature_4
0,Y,9.59,4.47,16.920000,59.520000,7.13,653.409973,68.849998,647.98999,55.400002,...,59.299999,276.500000,274.700012,274.799988,269.200012,255.000000,229.699997,66.300003,24.799999,27.500000
1,Y,9.60,4.48,16.910000,59.580002,7.13,653.409973,68.839996,647.98999,55.299999,...,59.299999,276.200012,275.500000,275.299988,270.799988,254.699997,229.500000,67.199997,24.799999,27.600000
2,Y,9.60,4.48,16.910000,59.580002,7.13,653.409973,68.839996,647.98999,55.299999,...,59.299999,276.200012,275.500000,275.299988,270.799988,254.699997,229.500000,67.199997,24.799999,27.600000
3,Y,9.59,4.48,16.910000,59.560001,7.13,653.419983,68.839996,647.98999,55.299999,...,59.299999,276.500000,275.000000,275.399994,271.100006,254.899994,230.000000,66.900002,25.000000,27.600000
4,Y,9.59,4.48,16.910000,59.560001,7.13,653.419983,68.839996,647.98999,55.299999,...,59.299999,276.500000,275.000000,275.399994,271.100006,254.899994,230.000000,66.900002,25.000000,27.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7991,Y,9.60,4.48,16.620001,59.560001,7.11,653.429993,68.330002,647.98999,54.900002,...,59.900002,276.500000,274.899994,275.100006,269.500000,255.600006,229.600006,66.099998,21.000000,22.400000
7992,Y,9.60,4.48,16.650000,59.560001,7.11,653.429993,68.349998,647.98999,55.000000,...,59.900002,275.799988,275.299988,275.500000,270.700012,254.899994,230.000000,66.099998,21.000000,22.400000
7993,Y,9.60,4.48,16.650000,59.560001,7.11,653.429993,68.349998,647.98999,55.000000,...,59.900002,275.799988,275.299988,275.500000,270.700012,254.899994,230.000000,66.099998,21.000000,22.400000
7994,Y,9.60,4.48,16.629999,59.580002,7.11,653.429993,68.330002,647.98999,54.900002,...,59.900002,276.100006,275.799988,275.000000,271.299988,255.100006,230.199997,65.199997,21.000000,22.299999


In [7]:
labeled_cn7["PassOrFail"] = labeled_cn7["PassOrFail"].replace('Y',0).replace('N',1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_17632\1196971330.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labeled_cn7["PassOrFail"] = labeled_cn7["PassOrFail"].replace('Y',0).replace('N',1)


In [8]:
### Clamp_Open_Position에서 눈에띄게 낮은 값들 제거
cn7 = labeled_cn7[labeled_cn7["Clamp_Open_Position"]>=200] 

In [9]:
cn7 = cn7.drop(columns=["Clamp_Open_Position"])

In [10]:
def remove_outliers(df, column):
    # 1사분위수(Q1)와 3사분위수(Q3) 계산
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    
    # IQR 계산
    IQR = Q3 - Q1
    
    # 이상치 범위 설정
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # 이상치 제거
    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    
    return filtered_df

In [11]:
cn7_Y = cn7[cn7["PassOrFail"]==1].copy()
print("CN7의 양품 개수:", len(cn7_Y))
cn7_N = cn7[cn7["PassOrFail"]==0].copy()
print("CN7의 불량 개수:", len(cn7_N))

CN7의 양품 개수: 39
CN7의 불량 개수: 6661


In [12]:
tmp1 = cn7_Y.copy()
tmp1 = remove_outliers(tmp1, "Injection_Time")
tmp1 = remove_outliers(tmp1, "Plasticizing_Time")
tmp1 = remove_outliers(tmp1, "Cycle_Time")
tmp1 = remove_outliers(tmp1, "Max_Injection_Speed")
tmp1 = remove_outliers(tmp1, "Mold_Temperature_3")
cn7_pass_removed = tmp1.copy()
cn7_pass_removed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17 entries, 47 to 7564
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   PassOrFail                17 non-null     int64  
 1   Injection_Time            17 non-null     float64
 2   Filling_Time              17 non-null     float64
 3   Plasticizing_Time         17 non-null     float64
 4   Cycle_Time                17 non-null     float64
 5   Clamp_Close_Time          17 non-null     float64
 6   Cushion_Position          17 non-null     float64
 7   Plasticizing_Position     17 non-null     float64
 8   Max_Injection_Speed       17 non-null     float64
 9   Max_Screw_RPM             17 non-null     float64
 10  Average_Screw_RPM         17 non-null     float64
 11  Max_Injection_Pressure    17 non-null     float64
 12  Max_Switch_Over_Pressure  17 non-null     float64
 13  Max_Back_Pressure         17 non-null     float64
 14  Average_Back_P

In [13]:
cn7_removed = pd.concat([cn7_pass_removed, cn7_N], ignore_index=True)
cn7_removed["PassOrFail"].value_counts()

PassOrFail
0    6661
1      17
Name: count, dtype: int64

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.cluster import DBSCAN
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import classification_report, confusion_matrix

# 독립변수와 종속변수 분리 (cn7_removed는 미리 정의된 DataFrame이어야 함)
y = cn7_removed["PassOrFail"]
X = cn7_removed.drop(columns=["PassOrFail"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 데이터 정규화: 학습 데이터에 fit한 후 양쪽을 transform
ss = StandardScaler()
ss.fit(X_train)
X_train_scaled = ss.transform(X_train)
X_test_scaled  = ss.transform(X_test)

# 이상탐지 모델 리스트 (여기서는 4가지 모델을 사용)
# Local Outlier Factor는 test 데이터에 대해 예측하려면 novelty=True로 설정해야 합니다.
models = {
    "Isolation Forest": IsolationForest(random_state=42),
    "One-Class SVM": OneClassSVM(),
    "DBSCAN": DBSCAN(eps=0.5, min_samples=5),
    "Local Outlier Factor": LocalOutlierFactor(n_neighbors=20, novelty=True, contamination='auto')
}

results = []
for model_name, model in models.items():
    # DBSCAN은 predict()가 없으므로, fit_predict()를 사용 (전체 테스트 데이터에 대해 클러스터링 수행)
    if model_name == "DBSCAN":
        y_pred = model.fit_predict(X_test_scaled)
        # DBSCAN은 outlier에 -1, 정상에 클러스터 번호를 부여하므로,
        # -1이면 0(불량), 그 외에는 1(정상)으로 변환
        y_pred_converted = [0 if label == -1 else 1 for label in y_pred]
    else:
        # 이상탐지 모델(나머지)은 일반적으로 정상 데이터의 특성만 학습하므로
        # X_train_scaled로 모델을 학습한 후 X_test_scaled에 대해 predict
        model.fit(X_train_scaled)
        y_pred = model.predict(X_test_scaled)
        # Isolation Forest와 One-Class SVM, LOF는 정상: 1, 이상: -1 을 반환하므로, -1이면 0, 1이면 1로 변환
        y_pred_converted = [1 if label == 1 else 0 for label in y_pred]
    
    results.append((model_name, y_pred_converted))

# 평가 지표 계산 및 결과 출력
for model_name, y_pred in results:
    print(f"{model_name} 결과:")
    print(classification_report(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
    print("혼돈 행렬:")
    print(cm_df)
    print("\n")


Isolation Forest 결과:
              precision    recall  f1-score   support

           0       0.98      0.12      0.21      2000
           1       0.00      0.00      0.00         4

    accuracy                           0.12      2004
   macro avg       0.49      0.06      0.11      2004
weighted avg       0.98      0.12      0.21      2004

혼돈 행렬:
          Predicted 0  Predicted 1
Actual 0          240         1760
Actual 1            4            0


One-Class SVM 결과:
              precision    recall  f1-score   support

           0       1.00      0.48      0.65      2000
           1       0.00      0.00      0.00         4

    accuracy                           0.48      2004
   macro avg       0.50      0.24      0.32      2004
weighted avg       0.99      0.48      0.65      2004

혼돈 행렬:
          Predicted 0  Predicted 1
Actual 0          963         1037
Actual 1            4            0


DBSCAN 결과:
              precision    recall  f1-score   support

           0 

C:\Users\Admin\anaconda3\envs\dg_env_dl\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Admin\anaconda3\envs\dg_env_dl\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Admin\anaconda3\envs\dg_env_dl\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [15]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
# --- 오토인코더 모델 정의 ---
input_dim = X_train_scaled.shape[1]  # 특성 차원
encoding_dim = 10  # 압축할 차원 수 (원하는 값으로 조정 가능)

# 입력층 정의
input_layer = Input(shape=(input_dim,))
# 인코딩 층 (은닉층)
encoded = Dense(encoding_dim, activation='relu')(input_layer)
# 디코딩 층 (출력층)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

# 오토인코더 모델 생성 및 컴파일
autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')

# --- 오토인코더 모델 학습 ---
autoencoder.fit(X_train_scaled, X_train_scaled,
                epochs=100,
                batch_size=32,
                validation_split=0.2)

# --- 재구성 오차 계산을 통한 이상치 탐지 ---
# 테스트 데이터에 대한 재구성
X_test_pred = autoencoder.predict(X_test_scaled)
# 각 샘플별 재구성 오차(평균 제곱 오차) 계산
reconstruction_error = np.mean(np.square(X_test_scaled - X_test_pred), axis=1)

# 95% 분위수를 기준으로 이상치 판별 임계값 산출
threshold = np.percentile(reconstruction_error, 98)
print("이상치 판별 기준 (재구성 오차):", threshold)

# 재구성 오차가 임계값보다 큰 경우 이상치(불량)로 판단: 0 (불량), 그렇지 않으면 1 (정상)
y_pred = np.where(reconstruction_error > threshold, 0, 1)

# --- 평가 지표 출력 ---
print("Classification Report:")
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
print("혼돈 행렬:")
print(cm_df)

Epoch 1/100
117/117 [==============================] - 1s 3ms/step - loss: 1.1683 - val_loss: 1.0742
Epoch 2/100
117/117 [==============================] - 0s 1ms/step - loss: 0.9712 - val_loss: 0.9171
Epoch 3/100
117/117 [==============================] - 0s 1ms/step - loss: 0.8652 - val_loss: 0.8559
Epoch 4/100
117/117 [==============================] - 0s 1ms/step - loss: 0.8243 - val_loss: 0.8274
Epoch 5/100
117/117 [==============================] - 0s 1ms/step - loss: 0.8007 - val_loss: 0.8070
Epoch 6/100
117/117 [==============================] - 0s 1ms/step - loss: 0.7830 - val_loss: 0.7908
Epoch 7/100
117/117 [==============================] - 0s 1ms/step - loss: 0.7683 - val_loss: 0.7768
Epoch 8/100
117/117 [==============================] - 0s 1ms/step - loss: 0.7540 - val_loss: 0.7621
Epoch 9/100
117/117 [==============================] - 0s 1ms/step - loss: 0.7399 - val_loss: 0.7491
Epoch 10/100
117/117 [==============================] - 0s 1ms/step - loss: 0.7291 - val_lo

In [16]:
threshold = np.percentile(reconstruction_error, 99.5
                         )
print("이상치 판별 기준 (재구성 오차):", threshold)

# 재구성 오차가 임계값보다 큰 경우 이상치(불량)로 판단: 0 (불량), 그렇지 않으면 1 (정상)
y_pred = np.where(reconstruction_error > threshold, 0, 1)

# --- 평가 지표 출력 ---
print("Classification Report:")
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
print("혼돈 행렬:")
print(cm_df)

이상치 판별 기준 (재구성 오차): 1.7062880686058668
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.00      0.01      2000
           1       0.00      0.25      0.00         4

    accuracy                           0.00      2004
   macro avg       0.36      0.13      0.00      2004
weighted avg       0.73      0.00      0.01      2004

혼돈 행렬:
          Predicted 0  Predicted 1
Actual 0            8         1992
Actual 1            3            1


In [17]:
import numpy as np
import pandas as pd
from itertools import product
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.cluster import DBSCAN
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# 사전 준비: cn7_removed DataFrame에서 독립변수와 종속변수 분리
y = cn7_removed["PassOrFail"]
X = cn7_removed.drop(columns=["PassOrFail"])

# 학습/테스트 데이터 분리 및 정규화
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# 각 모델별 하이퍼파라미터 그리드 정의
param_grids = {
    "Isolation Forest": {
        "n_estimators": [50, 100, 200],
        "max_samples": ["auto", 0.8],
        "contamination": [0.1, 0.15, 0.2]
    },
    "One-Class SVM": {
        "kernel": ["rbf", "linear"],
        "gamma": ["scale", "auto", 0.1],
        "nu": [0.01, 0.05, 0.1, 0.2]
    },
    "DBSCAN": {
        "eps": [0.3, 0.5, 0.7],
        "min_samples": [5, 10]
    },
    "Local Outlier Factor": {
        "n_neighbors": [10, 20, 30],
        "contamination": [0.1, 0.15, 0.2]
    }
}

# 결과 저장용 변수
best_results = {}

# 각 모델별로 하이퍼파라미터 튜닝 수행
for model_name, grid in param_grids.items():
    best_f1 = -1  # F1 스코어 최대값 기준
    best_params = None
    best_report = None
    best_cm = None
    
    for params in ParameterGrid(grid):
        # 모델별 인스턴스 생성 시 파라미터 적용
        if model_name == "Isolation Forest":
            model = IsolationForest(random_state=42, **params)
            model.fit(X_train_scaled)
            y_pred = model.predict(X_test_scaled)
            # IsolationForest는 정상 1, 이상 -1 → 변환: 1->1, -1->0
            y_pred_converted = [1 if label == 1 else 0 for label in y_pred]
        
        elif model_name == "One-Class SVM":
            model = OneClassSVM(**params)
            model.fit(X_train_scaled)
            y_pred = model.predict(X_test_scaled)
            # One-Class SVM 역시 정상 1, 이상 -1 → 1: 정상, -1: 불량(0)
            y_pred_converted = [1 if label == 1 else 0 for label in y_pred]
        
        elif model_name == "Local Outlier Factor":
            # novelty=True로 설정해야 테스트 데이터에 대해 예측 가능
            model = LocalOutlierFactor(novelty=True, **params)
            model.fit(X_train_scaled)
            y_pred = model.predict(X_test_scaled)
            y_pred_converted = [1 if label == 1 else 0 for label in y_pred]
        
        elif model_name == "DBSCAN":
            # DBSCAN은 fit_predict()를 사용
            model = DBSCAN(**params)
            y_pred = model.fit_predict(X_test_scaled)
            # DBSCAN은 -1을 이상(불량:0)으로, 그 외 클러스터 번호를 정상(1)으로 간주
            y_pred_converted = [0 if label == -1 else 1 for label in y_pred]
        
        # F1 스코어 계산 (여기서는 정상 클래스(1)를 양성으로 지정)
        current_f1 = f1_score(y_test, y_pred_converted, average="binary", pos_label=1)
        if current_f1 > best_f1:
            best_f1 = current_f1
            best_params = params
            best_report = classification_report(y_test, y_pred_converted)
            cm = confusion_matrix(y_test, y_pred_converted)
            best_cm = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
            
    best_results[model_name] = {
        "Best F1-Score": best_f1,
        "Best Hyperparameters": best_params,
        "Classification Report": best_report,
        "Confusion Matrix": best_cm
    }

# 최적의 모델별 결과 출력
for model_name, result in best_results.items():
    print(f"==== {model_name} ====")
    print("최적의 하이퍼파라미터:", result["Best Hyperparameters"])
    print("최고 F1-Score:", result["Best F1-Score"])
    print("\n[Classification Report]")
    print(result["Classification Report"])
    print("[Confusion Matrix]")
    print(result["Confusion Matrix"])
    print("\n")


C:\Users\Admin\anaconda3\envs\dg_env_dl\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Admin\anaconda3\envs\dg_env_dl\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Admin\anaconda3\envs\dg_env_dl\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

==== Isolation Forest ====
최적의 하이퍼파라미터: {'contamination': 0.1, 'max_samples': 'auto', 'n_estimators': 50}
최고 F1-Score: 0.0

[Classification Report]
              precision    recall  f1-score   support

           0       0.98      0.10      0.18      2000
           1       0.00      0.00      0.00         4

    accuracy                           0.10      2004
   macro avg       0.49      0.05      0.09      2004
weighted avg       0.98      0.10      0.18      2004

[Confusion Matrix]
          Predicted 0  Predicted 1
Actual 0          203         1797
Actual 1            4            0


==== One-Class SVM ====
최적의 하이퍼파라미터: {'gamma': 'scale', 'kernel': 'linear', 'nu': 0.2}
최고 F1-Score: 0.007476635514018692

[Classification Report]
              precision    recall  f1-score   support

           0       1.00      0.47      0.64      2000
           1       0.00      1.00      0.01         4

    accuracy                           0.47      2004
   macro avg       0.50      0.73  